<a href="https://colab.research.google.com/github/Jerathaditya/TCS-iON/blob/main/Svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

from keras.datasets import fashion_mnist

from keras.utils.np_utils import to_categorical

(trainX,trainy),(testX,testy)=fashion_mnist.load_data()

def load_dataset():
    (trainX,trainy),(testX,testy)=fashion_mnist.load_data()
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    return trainX, trainy, testX, testy

seed = 9
from sklearn.model_selection import StratifiedShuffleSplit

data_split = StratifiedShuffleSplit(test_size = 0.5, random_state = seed)
for train_index, test_index in data_split.split(trainX, trainy):
    split_data_92, split_data_8 = trainX[train_index], trainX[test_index]
    split_label_92, split_label_8 = trainy[train_index], trainy[test_index]
train_test_split = StratifiedShuffleSplit(test_size = 0.3, random_state = seed)

for train_index, test_index in train_test_split.split(split_data_8, split_label_8):
    train_data_70, test_data_30 = split_data_8[train_index], split_data_8[test_index]
    train_label_70, test_label_30 = split_label_8[train_index], split_label_8[test_index]

train_data = train_data_70
train_label = train_label_70

test_data = test_data_30
test_label = test_label_30

print('train_data : ', train_data.shape)
print('train_label : ', train_label.shape)
print('test_data : ', test_data.shape)
print('test_label : ', test_label.shape)

train_data= train_data.astype('float64') / 225.0
test_data = test_data.astype('float64') / 225.0

print('train_data : ', train_data.shape)
print('test_data : ', test_data.shape)

train_data_flat = train_data.reshape(train_data.shape[0], -1)
test_data_flat = test_data.reshape(test_data.shape[0], -1)

print('train_data_flat : ', train_data_flat.shape)
print('test_data_flat : ', test_data_flat.shape)

train_data_flat_t = train_data_flat
test_data_flat_t = test_data_flat

from sklearn.decomposition import PCA
train_data_pca = PCA(n_components = min(train_data_flat_t.shape)).fit_transform(train_data_flat_t)
test_data_pca = PCA(n_components = min(test_data_flat_t.shape)).fit_transform(test_data_flat_t)

print(train_data_pca.shape)
print(test_data_pca.shape)



from skimage import color
def svdFeatures(input_data):
    svdArray_input_data = []
    size = input_data.shape[0]
    for i in range(0, size):
#         img = color.rgb2gray(input_data[i])
        U, s, V = np.linalg.svd(input_data[i], full_matrices = False)
        S = [s[i] for i in range(28)]
        svdArray_input_data.append(s)
        svdMatrix_input_data = np.matrix(svdArray_input_data)
    return svdMatrix_input_data

train_data_svd = svdFeatures(train_data)
test_data_svd = svdFeatures(test_data)

print(train_data_svd.shape)
print(test_data_svd.shape)
        
        
    

from sklearn import svm
from sklearn.metrics import confusion_matrix

def train_test(train_dataset, test_dataset):
    
    clf = svm.SVC(gamma = 0.001, probability = True, random_state = 0, kernel = 'linear' )
    clf.fit(train_dataset, train_label)
    
    predicted = clf.predict(test_dataset)
    score = clf.score(test_dataset, test_label)
    print('score : ', score)
    conf_matrix = confusion_matrix(test_label, predicted)
    print('Confusion Matrix: ', conf_matrix)
       
train_test(train_data_pca, test_data_pca)
train_test(train_data_flat_t, test_data_flat_t)

train_test(train_data_svd, test_data_svd) 






train_data :  (21000, 28, 28)
train_label :  (21000,)
test_data :  (9000, 28, 28)
test_label :  (9000,)
train_data :  (21000, 28, 28)
test_data :  (9000, 28, 28)
train_data_flat :  (21000, 784)
test_data_flat :  (9000, 784)
(21000, 784)
(9000, 784)
(21000, 28)
(9000, 28)
score :  0.318
Confusion Matrix:  [[570  17  28  73  21   3 152   7  14  15]
 [ 55 625  14 189   6   1   3   0   7   0]
 [ 74   5 122  16 122  25  99  17 293 127]
 [258 150  65 247 135   2  39   1   1   2]
 [ 66  29 123  38 163  13 172   8 227  61]
 [  6   0   8   0   8 261  17 451  44 105]
 [231  13  70  32  75  20 212  22 111 114]
 [  0   0   3   0   1 173   1 461 223  38]
 [ 26  14  87   5   9 144 146  31 196 242]
 [  0   0 130   0  83 360 153  18 151   5]]
score :  0.838
Confusion Matrix:  [[713   3  24  52   4   1  93   0  10   0]
 [  6 872   3  18   0   0   0   0   1   0]
 [ 25   3 650   6 113   1  97   0   5   0]
 [ 56  22  26 749  32   0  12   0   3   0]
 [  4   3  87  30 700   0  71   0   5   0]
 [  0   0   1 